In [1]:
# !pip install torch
# !pip install torchvision

In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [4]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
# train_data.data.shape
train_data.data.size()

torch.Size([60000, 28, 28])

In [7]:
train_data.targets.size()

torch.Size([60000])

In [8]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [9]:
from torch.utils.data import DataLoader

loaders = {
    'train' : DataLoader(train_data,
                        batch_size = 100,
                        shuffle = True,
                        num_workers = 1),
    
    'test' : DataLoader(train_data,
                        batch_size = 100,
                        shuffle = True,
                        num_workers = 1),    
    
}

In [10]:
# get the data into our model

loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1ca880969b0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x1ca88097c70>}

In [11]:
# Find model architecture, deep learning neural network structure

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fcl = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fcl(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        
        return F.softmax(x, dim = 1)

In [12]:
# For optimize and train

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)

# Define a lost function

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate (loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)/len(loaders["train"].dataset)}({100. * batch_idx/ len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


            
def test():
    model.eval()
    
    test_loss = 0 
    correct = 0
    
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim = 1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            
    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')

In [13]:
# Start the training
for epoch in range(1, 11):
    train(epoch)
    test()

Train Epoch: 1 [0.0(0%)]	2.301236
Train Epoch: 1 [0.03333333333333333(3%)]	2.288034
Train Epoch: 1 [0.06666666666666667(7%)]	2.188403
Train Epoch: 1 [0.1(10%)]	2.062964
Train Epoch: 1 [0.13333333333333333(13%)]	2.007261
Train Epoch: 1 [0.16666666666666666(17%)]	1.880702
Train Epoch: 1 [0.2(20%)]	1.861166
Train Epoch: 1 [0.23333333333333334(23%)]	1.739914
Train Epoch: 1 [0.26666666666666666(27%)]	1.778570
Train Epoch: 1 [0.3(30%)]	1.719941
Train Epoch: 1 [0.3333333333333333(33%)]	1.801684
Train Epoch: 1 [0.36666666666666664(37%)]	1.700402
Train Epoch: 1 [0.4(40%)]	1.680130
Train Epoch: 1 [0.43333333333333335(43%)]	1.660062
Train Epoch: 1 [0.4666666666666667(47%)]	1.616022
Train Epoch: 1 [0.5(50%)]	1.676103
Train Epoch: 1 [0.5333333333333333(53%)]	1.678331
Train Epoch: 1 [0.5666666666666667(57%)]	1.648888
Train Epoch: 1 [0.6(60%)]	1.628299
Train Epoch: 1 [0.6333333333333333(63%)]	1.641236
Train Epoch: 1 [0.6666666666666666(67%)]	1.672595
Train Epoch: 1 [0.7(70%)]	1.640128
Train Epoch: 1 

Train Epoch: 6 [0.8333333333333334(83%)]	1.523556
Train Epoch: 6 [0.8666666666666667(87%)]	1.492946
Train Epoch: 6 [0.9(90%)]	1.507527
Train Epoch: 6 [0.9333333333333333(93%)]	1.474783
Train Epoch: 6 [0.9666666666666667(97%)]	1.520185

Test set: Average loss: 0.0149, Accuracy 58062/60000 (97%
)
Train Epoch: 7 [0.0(0%)]	1.503155
Train Epoch: 7 [0.03333333333333333(3%)]	1.541767
Train Epoch: 7 [0.06666666666666667(7%)]	1.603888
Train Epoch: 7 [0.1(10%)]	1.521546
Train Epoch: 7 [0.13333333333333333(13%)]	1.533028
Train Epoch: 7 [0.16666666666666666(17%)]	1.534083
Train Epoch: 7 [0.2(20%)]	1.517006
Train Epoch: 7 [0.23333333333333334(23%)]	1.510044
Train Epoch: 7 [0.26666666666666666(27%)]	1.526067
Train Epoch: 7 [0.3(30%)]	1.508669
Train Epoch: 7 [0.3333333333333333(33%)]	1.517626
Train Epoch: 7 [0.36666666666666664(37%)]	1.557824
Train Epoch: 7 [0.4(40%)]	1.499905
Train Epoch: 7 [0.43333333333333335(43%)]	1.507748
Train Epoch: 7 [0.4666666666666667(47%)]	1.528217
Train Epoch: 7 [0.5(50%)

In [14]:
device

device(type='cpu')

In [ ]:
import matplotlib.pyplot as plt

model.eval()

data, target = test_data[3]

data = data.unsqueeze(0).to(device)

output = model(data)

prediction = output.argmax(dim = 1, keepdim = True).item()

print(f'Prediction: {prediction}')

image = data.squeeze(0).squeeze(0).cpu().numpy()

plt.imshow(image, cmap = 'gray')

plt.show()


Prediction: 0


In [ ]:
# written by Foroogh Behroozi